## 载入数据

把我们在write_gap里导出的特征向量加载进来，并按不同组合去合成特征向量，作为后续模型的输入。

In [ ]:
import h5py
import numpy as np
from sklearn.utils import shuffle

X_train = []
X_test = []

for filename in ["gap_f_ResNet50.h5","gap_f_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

## 构建模型

这里构建一个简单分类模型，利用上面出的X_train去训练模型。该模型较简单，可以快速完成训练。

In [21]:
from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, batch_size=128, nb_epoch=15, validation_split=0.2)

/opt/anaconda3/envs/mlnd-dl/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 20000 samples, validate on 5000 samples
Epoch 1/15
20000/20000 [==============================] - 0s - loss: 2.2157 - acc: 0.7552 - val_loss: 0.1101 - val_acc: 0.9730
Epoch 2/15
20000/20000 [==============================] - 0s - loss: 0.2729 - acc: 0.9491 - val_loss: 0.0282 - val_acc: 0.9940
Epoch 3/15
20000/20000 [==============================] - 0s - loss: 0.0850 - acc: 0.9798 - val_loss: 0.0752 - val_acc: 0.9822
Epoch 4/15
20000/20000 [==============================] - 0s - loss: 0.0476 - acc: 0.9876 - val_loss: 0.0274 - val_acc: 0.9944
Epoch 5/15
20000/20000 [==============================] - 0s - loss: 0.0414 - acc: 0.9899 - val_loss: 0.0457 - val_acc: 0.9902
Epoch 6/15
20000/20000 [==============================] - 0s - loss: 0.0359 - acc: 0.9928 - val_loss: 0.0251 - val_acc: 0.9956
Epoch 7/15
20000/20000 [==============================] - 0s - loss: 0.0310 - acc: 0.9926 - val_loss: 0.0418 - val_acc: 0.9896
Epoch 8/15
20000/20000 [==============================] - 0s -

In [17]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

11456/12500 [==========================>...] - ETA: 0s

## 输出结果

把预测的结果按照要求格式输出。

In [18]:
import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred_02.csv', index=None)
df.head(10)

Found 12500 images belonging to 1 classes.


,id,label
0,1,9.999993e-01
1,2,9.999828e-01
2,3,9.999940e-01
3,4,9.999833e-01
4,5,1.707789e-05
5,6,4.399886e-08
6,7,3.799499e-06
7,8,6.051049e-08
8,9,6.206559e-06
9,10,6.153911e-07
